In [1540]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

In [1541]:
qs_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508694016501'
qs_r = requests.get(qs_url)

In [1542]:
qs_uni_dict = qs_r.json()

create the data frame

In [1543]:
qs_columns = ['Name','Region','Location','QS Rank','QS Score','QS No Staffs', 'QS No International Staffs',
              'QS No Students', 'QS No International Students']
qs_uni_rank = pd.DataFrame(index = [], columns=qs_columns)
qs_uni_rank.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students


In [1544]:
def get_numbers_stud_staff(url):
    r_uni = requests.get("https://www.topuniversities.com" + url)
    
    html_request_uni = BeautifulSoup(r_uni.text, 'html.parser')
    
    header_staff = html_request_uni.find('div',class_='faculty-main wrapper col-md-4')
    header_stud_total = html_request_uni.find('div',class_='students-main wrapper col-md-4')
    header_stud_inter = html_request_uni.find('div',class_='int-students-main wrapper col-md-4')
    
    if ((header_staff == None) & (header_stud_total == None) & (header_stud_inter == None)):
        return (-1,-1,-1,-1)
        

    subdiv_staff_total = header_staff.find('div', class_='total faculty')
    subdiv_staff_inter = header_staff.find('div', class_= 'inter faculty')
    subdiv_stud_total = header_stud_total.find('div', class_='total student')
    subdiv_stud_inter = header_stud_inter.find('div', class_='total inter')
    
    num_staff_total = int(subdiv_staff_total.find('div', class_ = 'number').text[1:-1].replace(',',''))
    if (subdiv_staff_inter != None):
        num_staff_inter = int(subdiv_staff_inter.find('div', class_ = 'number').text[1:-1].replace(',',''))
    else:
        num_staff_inter = -1    
    num_stud_total = int(subdiv_stud_total.find('div', class_ = 'number').text[1:-1].replace(',',''))
    num_stud_inter = int(subdiv_stud_inter.find('div', class_ = 'number').text[1:-1].replace(',',''))
    
    return (num_staff_total, num_staff_inter, num_stud_total, num_stud_inter)

In [1545]:
uni_list = qs_uni_dict['data']
i = 0
for uni in uni_list[:200]:
    i+=1
    if (i%40 == 0):
        print(i)
    (tot_staff, inter_staff, tot_stud, inter_stud) = get_numbers_stud_staff(uni['url'])
    uni_df = pd.DataFrame([[uni['title'],uni['region'], uni['country'], uni['rank_display'].replace('=',' '),
                            uni['score'], tot_staff, inter_staff, tot_stud, inter_stud]], columns = qs_columns)
    qs_uni_rank = qs_uni_rank.append(uni_df)

qs_uni_rank.index = list(range(1,201))


40
80
120
160
200


# Second Website: THE ranking

Note that as the number students, the number of staffs and the number of international students are already mentionned in the QS ranking website they are not scrapped from the Times University Ranking website. We indeed assume that the information provided by the QS ranking website are reliable.

In [1546]:
the_r = requests.get("https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json")

the_uni_dict = the_r.json()['data'][:200]

the_columns = ['Name', 'Location', 'THE Rank', 'THE Score', 'THE No Students', 'THE International Students Ratio','THE No Students per Staff']
#the_columns = ['Name','Location','THE Rank','Score', 'No students', 'International students ratio','No students per staff']
the_uni_rank = pd.DataFrame(index = [], columns=the_columns)
the_uni_rank.head()

,Name,Location,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff


In [1547]:
for uni in the_uni_dict:
    uni_df = pd.DataFrame([[uni['name'],uni['location'], uni['rank'],uni['scores_overall'],
                            uni['stats_number_students'],uni['stats_pc_intl_students'],
                          uni['stats_student_staff_ratio']]],columns = the_columns)
    the_uni_rank = the_uni_rank.append(uni_df)
    
the_uni_rank.index = list(range(1,201))

In [1548]:
print(qs_columns)
print(the_columns)

['Name', 'Region', 'Location', 'QS Rank', 'QS Score', 'QS No Staffs', 'QS No International Staffs', 'QS No Students', 'QS No International Students']
['Name', 'Location', 'THE Rank', 'THE Score', 'THE No Students', 'THE International Students Ratio', 'THE No Students per Staff']


###### Printing the two DataFrames

In [1549]:
qs_uni_rank.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students
1,Massachusetts Institute of Technology (MIT),North America,United States,1,100,2982,1679,11067,3717
2,Stanford University,North America,United States,2,98.7,4285,2042,15878,3611
3,Harvard University,North America,United States,3,98.4,4350,1311,22429,5266
4,California Institute of Technology (Caltech),North America,United States,4,97.7,953,350,2255,647
5,University of Cambridge,Europe,United Kingdom,5,95.6,5490,2278,18770,6699


In [1550]:
the_uni_rank.head()

,Name,Location,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff
1,University of Oxford,United Kingdom,1,94.3,"20,409",38%,11.2
2,University of Cambridge,United Kingdom,2,93.2,"18,389",35%,10.9
3,California Institute of Technology,United States,=3,93.0,"2,209",27%,6.5
4,Stanford University,United States,=3,93.0,"15,845",22%,7.5
5,Massachusetts Institute of Technology,United States,5,92.5,"11,177",34%,8.7


In [1551]:
qs_uni_rank.to_csv("qs_uni_rank.csv")

In [1552]:
the_uni_rank.to_csv("the_uni_rank.csv")

# Merging the two DataFrames

In [1553]:
import pandas as pd
import math

In [1554]:
qs_uni_rank = pd.DataFrame.from_csv('qs_uni_rank.csv')
the_uni_rank = pd.DataFrame.from_csv('the_uni_rank.csv')

## First filtering

##### Filtering QS data
###### Please run this cell several times until the warning disappears

In [1564]:
# Removing brackets and choosing the option that is less likely to be an acronym
for ind_qs, qs_uni_name in enumerate(qs_uni_rank['Name']):
    #print(type(qs_uni_name))
    if ('(' in qs_uni_name):
        first_brack = qs_uni_name.index('(')
        second_brack = qs_uni_name.index(')')
        brack_content = qs_uni_name[first_brack+1:second_brack]
        if (' ' in brack_content):
            qs_uni_rank['Name'].iloc[ind_qs] = brack_content
        else:
            qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name[:(first_brack-1)]
    
    if ('é' in qs_uni_name):
        qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name.replace('é','e')
    if ('ä' in qs_uni_name):
        qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name.replace('ä','ae')
    if ('ü' in qs_uni_name):
        qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name.replace('ü','u')
    if ('ó' in qs_uni_name):
        qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name.replace('ó','o')
    if ('É' in qs_uni_name):
        qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name.replace('É','E')
    
    if ('The ' == qs_uni_name[:4]):
        qs_uni_rank['Name'].iloc[ind_qs] = qs_uni_name[4:]

In [1565]:
qs_uni_rank.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students
1,Massachusetts Institute of Technology,North America,United States,1,100.0,2982,1679,11067,3717
2,Stanford University,North America,United States,2,98.7,4285,2042,15878,3611
3,Harvard University,North America,United States,3,98.4,4350,1311,22429,5266
4,California Institute of Technology,North America,United States,4,97.7,953,350,2255,647
5,University of Cambridge,Europe,United Kingdom,5,95.6,5490,2278,18770,6699


###### Filtering THE data
###### Please run this cell several times until the warning disappears

In [1569]:
# Removing brackets and choosing the option that is less likely to be an acronym
for ind_the, the_uni_name in enumerate(the_uni_rank['Name']):
    if ('(' in the_uni_name):
        first_brack = the_uni_name.index('(')
        second_brack = the_uni_name.index(')')
        brack_content = the_uni_name[first_brack+1:second_brack]
        
        if (' ' in brack_content):
            the_uni_rank['Name'].iloc[ind_the] = brack_content
        else:
            the_uni_rank['Name'].iloc[ind_the] = the_uni_name[:first_brack-1]
    
    if ('é' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('é','e')
    if ('ä' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('ä','ae')
    if ('ü' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('ü','u')
    if ('ó' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('ó','o')
    if ('É' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('É','E')
    if ('’' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('’','\'')
    if ('–' in the_uni_name):
        the_uni_rank['Name'].iloc[ind_the] = the_uni_name.replace('–', '-')


In [1570]:
the_uni_rank.head()

,Name,Location,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff
1,University of Oxford,United Kingdom,1,94.3,"20,409",38%,11.2
2,University of Cambridge,United Kingdom,2,93.2,"18,389",35%,10.9
3,California Institute of Technology,United States,=3,93.0,"2,209",27%,6.5
4,Stanford University,United States,=3,93.0,"15,845",22%,7.5
5,Massachusetts Institute of Technology,United States,5,92.5,"11,177",34%,8.7


There is only one difference in the location definition between both DataFrames that is fixed now. Indeed it is necessary to do so for the merging operation to not fail.

In [1571]:
print("Before changing:")
print(qs_uni_rank[qs_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])
print(the_uni_rank[the_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])
the_uni_rank.loc[194, 'Location'] = 'Russia'
print()
print("After changing:")
print(the_uni_rank[the_uni_rank['Name'] == 'Lomonosov Moscow State University']['Location'])


Before changing:
95    Russia
Name: Location, dtype: object
194    Russia
Name: Location, dtype: object

After changing:
194    Russia
Name: Location, dtype: object


In [1572]:
first_merged_uni_rank = pd.merge(qs_uni_rank, the_uni_rank, how = "outer")
#merged_uni_rank = merged_uni_rank.set_index('Name')
first_merged_uni_rank.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff
0,Massachusetts Institute of Technology,North America,United States,1.0,100.0,2982.0,1679.0,11067.0,3717.0,5,92.5,"11,177",34%,8.7
1,Stanford University,North America,United States,2.0,98.7,4285.0,2042.0,15878.0,3611.0,=3,93.0,"15,845",22%,7.5
2,Harvard University,North America,United States,3.0,98.4,4350.0,1311.0,22429.0,5266.0,6,91.8,"20,326",26%,8.9
3,California Institute of Technology,North America,United States,4.0,97.7,953.0,350.0,2255.0,647.0,=3,93.0,"2,209",27%,6.5
4,University of Cambridge,Europe,United Kingdom,5.0,95.6,5490.0,2278.0,18770.0,6699.0,2,93.2,"18,389",35%,10.9


# Second filtering
We construct two new DataFrames for the universities that didn't match by name during the first filtering. These two DataFrames are named ´qs_unmatched´ and ´the_unmatched´ respectively for the universities of the QS ranking and of the THE ranking that didn't match by name any counter part. Then we create a new DataFrame for the universities that perfectly matched by name which is called ´perfect_match_merged´.

In [1573]:
qs_unmatched = first_merged_uni_rank[first_merged_uni_rank['THE No Students'].isnull()].copy()
the_unmatched = first_merged_uni_rank[first_merged_uni_rank['QS No Students'].isnull()].copy()
perfect_match_merged_bis = first_merged_uni_rank[first_merged_uni_rank['THE No Students'].notnull()].copy()
perfect_match_merged = perfect_match_merged_bis[perfect_match_merged_bis['QS No Students'].notnull()].copy()

In [1574]:
qs_unmatched.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff
9,ETH Zurich - Swiss Federal Institute of Techno...,Europe,Switzerland,10.0,93.3,2477.0,1886.0,19815.0,7563.0,NaN,NaN,NaN,NaN,NaN
40,KAIST - Korea Advanced Institute of Science & ...,Asia,South Korea,41.0,80.4,1250.0,147.0,9826.0,584.0,NaN,NaN,NaN,NaN,NaN
42,"Ecole normale superieure, Paris",Europe,France,43.0,79.9,178.0,75.0,1907.0,374.0,NaN,NaN,NaN,NaN,NaN
44,UNSW Sydney,Oceania,Australia,45.0,78.9,2924.0,1612.0,39784.0,14292.0,NaN,NaN,NaN,NaN,NaN
55,Tokyo Institute of Technology,Asia,Japan,56.0,74.8,1563.0,191.0,9832.0,1071.0,NaN,NaN,NaN,NaN,NaN


In [1575]:
the_unmatched.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff
200,ETH Zurich - Swiss Federal Institute of Techno...,NaN,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,=10,87.7,"19,233",38%,14.6
201,LMU Munich,NaN,Germany,NaN,NaN,NaN,NaN,NaN,NaN,=34,76.2,"36,091",15%,15.6
202,Karolinska Institute,NaN,Sweden,NaN,NaN,NaN,NaN,NaN,NaN,=38,75.3,"7,424",19%,10.9
203,Heidelberg University,NaN,Germany,NaN,NaN,NaN,NaN,NaN,NaN,45,72.3,"28,876",17%,23.2
204,Washington University in St Louis,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,=50,70.8,"12,600",17%,7.5


In [1576]:
perfect_match_merged.head()

,Name,Region,Location,QS Rank,QS Score,QS No Staffs,QS No International Staffs,QS No Students,QS No International Students,THE Rank,THE Score,THE No Students,THE International Students Ratio,THE No Students per Staff
0,Massachusetts Institute of Technology,North America,United States,1.0,100.0,2982.0,1679.0,11067.0,3717.0,5,92.5,"11,177",34%,8.7
1,Stanford University,North America,United States,2.0,98.7,4285.0,2042.0,15878.0,3611.0,=3,93.0,"15,845",22%,7.5
2,Harvard University,North America,United States,3.0,98.4,4350.0,1311.0,22429.0,5266.0,6,91.8,"20,326",26%,8.9
3,California Institute of Technology,North America,United States,4.0,97.7,953.0,350.0,2255.0,647.0,=3,93.0,"2,209",27%,6.5
4,University of Cambridge,Europe,United Kingdom,5.0,95.6,5490.0,2278.0,18770.0,6699.0,2,93.2,"18,389",35%,10.9


In [1577]:
perfect_match_merged.isnull().sum()

Name                                0
Region                              0
Location                            0
QS Rank                             0
QS Score                            0
QS No Staffs                        0
QS No International Staffs          0
QS No Students                      0
QS No International Students        0
THE Rank                            0
THE Score                           0
THE No Students                     0
THE International Students Ratio    0
THE No Students per Staff           0
dtype: int64

In [1578]:
qs_unmatched.shape

(61, 14)

In [1579]:
the_unmatched.shape

(61, 14)

In [1580]:
qs_unmatched['Name'].head()

9     ETH Zurich - Swiss Federal Institute of Techno...
40    KAIST - Korea Advanced Institute of Science & ...
42                      Ecole normale superieure, Paris
44                                          UNSW Sydney
55                        Tokyo Institute of Technology
Name: Name, dtype: object

In [1581]:
the_unmatched['Name'].head()

200    ETH Zurich - Swiss Federal Institute of Techno...
201                                           LMU Munich
202                                 Karolinska Institute
203                                Heidelberg University
204                    Washington University in St Louis
Name: Name, dtype: object

###### Collecting unmatched university names
The names are then going to be filtered in order to remove their stop words. This way they will be more easily comparable.

In [1582]:
names_qs_unmatched = [qs_unmatched['Name'].iloc[x] for x in range(qs_unmatched.shape[0])]
names_the_unmatched = [the_unmatched['Name'].iloc[x] for x in range(the_unmatched.shape[0])]

In [1583]:
print(names_qs_unmatched)
print()
print(names_the_unmatched)

['ETH Zurich - Swiss Federal Institute of Technology', 'KAIST - Korea Advanced Institute of Science & Technology', 'Ecole normale superieure, Paris', 'UNSW Sydney', 'Tokyo Institute of Technology', 'Osaka University', 'Ludwig-Maximilians-Universitaet Munchen', 'Ruprecht-Karls-Universitaet Heidelberg', 'Pohang University of Science And Technology', 'Universidad de Buenos Aires', 'Tohoku University', 'University of North Carolina, Chapel Hill', 'Trinity College Dublin, The University of Dublin', 'Korea University', 'Washington University in St. Louis', 'Yonsei University', 'KIT, Karlsruhe Institute of Technology', 'Universiti Malaya', 'Nagoya University', 'Humboldt-Universitaet zu Berlin', 'Universidade de São Paulo', 'Universidad Nacional Autonoma de Mexico ', 'Hokkaido University', 'Wageningen University', 'Freie Universitaet Berlin', 'Kyushu University', 'Universite de Montreal', 'Universite Pierre et Marie Curie', 'Chalmers University of Technology', 'Pontificia Universidad Catolica 

###### Removing of stop words
Below is defined the dictionnary of words and characters that are considered as stop words. They are removed from both university names list thanks to the ´remove_stop_word´ function defined below.

In [1584]:
stop_words_repo = [' & ', ' de ', ' of ', ',', ' di ', '-', '\'', ' and ', ' And ', '.', ' zu ', ' at ', ' et ']

In [1585]:
def remove_stop_words(string, stop_words_repo):
    for stop_word in stop_words_repo:
        while (stop_word in string):
            index = string.find(stop_word)            
            string = string[:index] + string[(index + len(stop_word)):]
    return string

In [1586]:
remove_stop_words('University de Lausanne', stop_words_repo)

'UniversityLausanne'

Removing of stop wors for the QS unmatched university names.

In [1587]:
for i, name in enumerate(names_qs_unmatched):
    new_name = remove_stop_words(name, stop_words_repo)
    # remove spaces
    new_name = remove_stop_words(new_name, ' ')
    names_qs_unmatched[i] = new_name
print(names_qs_unmatched)

['ETHZurichSwissFederalInstituteTechnology', 'KAISTKoreaAdvancedInstituteScienceTechnology', 'EcolenormalesuperieureParis', 'UNSWSydney', 'TokyoInstituteTechnology', 'OsakaUniversity', 'LudwigMaximiliansUniversitaetMunchen', 'RuprechtKarlsUniversitaetHeidelberg', 'PohangUniversityScienceTechnology', 'UniversidadBuenosAires', 'TohokuUniversity', 'UniversityNorthCarolinaChapelHill', 'TrinityCollegeDublinTheUniversityDublin', 'KoreaUniversity', 'WashingtonUniversityinStLouis', 'YonseiUniversity', 'KITKarlsruheInstituteTechnology', 'UniversitiMalaya', 'NagoyaUniversity', 'HumboldtUniversitaetBerlin', 'UniversidadeSãoPaulo', 'UniversidadNacionalAutonomaMexico', 'HokkaidoUniversity', 'WageningenUniversity', 'FreieUniversitaetBerlin', 'KyushuUniversity', 'UniversiteMontreal', 'UniversitePierreMarieCurie', 'ChalmersUniversityTechnology', 'PontificiaUniversidadCatolicaChile', 'TUBerlin', 'HebrewUniversityJerusalem', 'UniversityOtago', 'UniversityWaterloo', 'UniversitecatholiqueLouvain', 'Hanyan

Removing of stop wors for the THE unmatched university names.

In [1588]:
'UniversityBonn'

'UniversityBonn'

In [1589]:
for i, name in enumerate(names_the_unmatched):
    new_name = remove_stop_words(name, stop_words_repo)
    # remove spaces
    new_name = remove_stop_words(new_name, ' ')
    names_the_unmatched[i] = new_name
print(names_the_unmatched)

['ETHZurichSwissFederalInstituteTechnologyZurich', 'LMUMunich', 'KarolinskaInstitute', 'HeidelbergUniversity', 'WashingtonUniversityinStLouis', 'UniversityNorthCarolinaChapelHill', 'HumboldtUniversityBerlin', 'WageningenUniversityResearch', 'ParisSciencesLettresPSLResearchUniversityParis', 'UniversityFreiburg', 'UniversityNewSouthWales', 'FreeUniversityBerlin', 'TechnicalUniversityBerlin', 'UniversityTubingen', 'KoreaAdvancedInstituteScienceTechnology', 'UniversityBonn', 'VanderbiltUniversity', 'UniversityMontreal', 'IndianaUniversity', 'TrinityCollegeDublin', 'RadboudUniversityNijmegen', 'GeorgetownUniversity', 'PierreMarieCurieUniversity', 'UniversityMannheim', 'ArizonaStateUniversity', 'ChariteUniversitaetsmedizinBerlin', 'UniversiteCatholiqueLouvain', 'KarlsruheInstituteTechnology', 'PohangUniversityScienceTechnology', 'PompeuFabraUniversity', 'UniversityCologne', 'AutonomousUniversityBarcelona', 'UniversitySussex', 'UniversityNotreDame', 'ScuolaSuperioreSantAnna', 'TUDresden', 'Ul

###### Perfect match check after stop words filtering
We now compare the university names without their stop words and we chack if there is any perfect match.

In [1590]:
count = 0
qs_matches = []
the_matches = []
for ind_the, name_the in enumerate(names_the_unmatched):
    true_the_name = the_unmatched['Name'].iloc[ind_the]
    for ind_qs, name_qs in enumerate(names_qs_unmatched):
        if (name_qs == name_the):
            count+=1
            true_qs_name = qs_unmatched['Name'].iloc[ind_qs]
            try:
                the_unmatched['Name'].iloc[ind_the] = true_qs_name
            except SettingWithCopyWarning:
                print("catched")
            qs_matches+=[ind_qs]
            the_matches+=[ind_the]
            print(true_the_name, "changed to", true_qs_name, ind_the)
print("Number of university names changed:", count)


Washington University in St Louis changed to Washington University in St. Louis 4
University of North Carolina at Chapel Hill changed to University of North Carolina, Chapel Hill 5
Pohang University of Science and Technology changed to Pohang University of Science And Technology 28
Number of university names changed: 3


/home/adrien/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [1591]:
new_df_to_append = pd.merge(the_unmatched[the_columns].iloc[the_matches].copy(),\
                            qs_unmatched[qs_columns].iloc[qs_matches].copy())
perfect_match_merged = perfect_match_merged.append(new_df_to_append)

In [1592]:
perfect_match_merged.tail()

,Location,Name,QS No International Staffs,QS No International Students,QS No Staffs,QS No Students,QS Rank,QS Score,Region,THE International Students Ratio,THE No Students,THE No Students per Staff,THE Rank,THE Score
196,United States,Texas A&M University,206.0,4900.0,3446.0,60294.0,195.0,48.1,North America,9%,"56,245",23.6,=159,55.0
198,Netherlands,Maastricht University,502.0,8234.0,1277.0,16385.0,200.0,47.9,Europe,50%,"16,727",18.0,103,60.4
0,United States,Washington University in St. Louis,784.0,2706.0,2699.0,13474.0,100.0,64.6,North America,17%,"12,600",7.5,=50,70.8
1,United States,"University of North Carolina, Chapel Hill",524.0,1934.0,4083.0,25373.0,80.0,67.8,North America,8%,"34,684",9.7,=56,69.0
2,South Korea,Pohang University of Science And Technology,113.0,126.0,664.0,3117.0,71.0,69.4,Asia,4%,"3,046",10.4,=137,57.3


We drop the indices of the university names that matched in the two string lists that we are maintaining.

In [1593]:
names_qs_unmatched = [names_qs_unmatched[x] for x in range(len(names_qs_unmatched)) if x not in qs_matches]
names_the_unmatched = [names_the_unmatched[x] for x in range(len(names_the_unmatched)) if x not in the_matches]

We now drop the rows that matched from the two DataFrames of unmatched university names that we are maintaining.

In [1594]:
qs_unmatched = qs_unmatched.drop(qs_unmatched.index[qs_matches])
the_unmatched = the_unmatched.drop(the_unmatched.index[the_matches])

### Levenshtein distance computation
We actually end up with only one new match but we are going to use these lists of filtered names in the following.
Firstly we define the ´levenshtein_distance´ function which computes the levenshtein distance between two strings. See https://www.wikiwand.com/en/Levenshtein_distance for more details about this metrics.

In [1595]:
def levenshtein_distance(string1, string2):
    rows = len(string1)+1
    cols = len(string2)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    for i in range(1, rows):
        dist[i][0] = i
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if string1[row-1] == string2[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1, dist[row][col-1] + 1, dist[row-1][col-1] + cost)
 
    return dist[row][col]

###### Matches for Levenshtein distance less than 5
Below we try to find the universities in the unmatched ones that have the smallest Levenshtein distance between each others. We print each change that have been made and we store them in an array to be sure that no university name change have been made mistakenly. To avoid unconsistent matches we set a cut_off of 5. This way it prevents universities whose names are too different from matching.

In [1596]:
changes=[]
matched_qs=[]
matched_the=[]
cut_off = 5
for ind_the, name_the in enumerate(names_the_unmatched):
    min_dist = math.inf
    flag = False
    true_the_name = the_unmatched['Name'].iloc[ind_the]
    for ind_qs, name_qs in enumerate(names_qs_unmatched):
        lev_dist = levenshtein_distance(name_qs, name_the)
        if (lev_dist < min_dist and lev_dist < 5):
            flag = True
            best_qs = ind_qs
            true_qs_name = qs_unmatched['Name'].iloc[ind_qs]
            min_dist = lev_dist
            #the_unmatched['Name'].loc[ind_the] = true_qs_name
    if (flag == True):
        matched_qs += [m_qs]
        matched_the += [ind_the]
        print(true_the_name, 'with', true_qs_name)
        changes += [[(ind_the, true_the_name), (best_qs, true_qs_name)]]
changes

Humboldt University of Berlin with Humboldt-Universitaet zu Berlin
Free University of Berlin with Freie Universitaet Berlin
University of Bonn with University of Bath
University of Montreal with Universite de Montreal
Universite Catholique de Louvain with Universite catholique de Louvain
Karlsruhe Institute of Technology with KIT, Karlsruhe Institute of Technology
University of Miami with University of Otago


[[(4, 'Humboldt University of Berlin'),
  (16, 'Humboldt-Universitaet zu Berlin')],
 [(9, 'Free University of Berlin'), (21, 'Freie Universitaet Berlin')],
 [(13, 'University of Bonn'), (34, 'University of Bath')],
 [(15, 'University of Montreal'), (23, 'Universite de Montreal')],
 [(24, 'Universite Catholique de Louvain'),
  (31, 'Universite catholique de Louvain')],
 [(25, 'Karlsruhe Institute of Technology'),
  (13, 'KIT, Karlsruhe Institute of Technology')],
 [(48, 'University of Miami'), (29, 'University of Otago')]]

We can remark that 'University of Bonn', 'TU Dresden', 'University of Miami', 'University of Dundee' and 'Indiana University' from the THE unmatched database have been wrongly assigned. We are thus going to assign them manually to the ´perfect_match_merged´ DataFrame steming of the first filtering. Indeed as there is no better match in the QS database for these THE university names we will assume that they simply do not belong to the top 200 QS ranking. Note that this process of compting the Levenshtein distance to compare the different university names is very useful to identify the stop words that differentiate the two name versions of a same university. Thus iteratively we add the stop words to the ´stop_words_repo´ instantiated a short time ago.


In [1597]:
indices_the_matched = [4, 9, 15, 24, 25]
indices_qs_matched = [16, 21, 23, 31, 13]

new_the_names = qs_unmatched['Name'].iloc[indices_qs_matched]
for i, name in enumerate(new_the_names):
    the_unmatched['Name'].iloc[indices_the_matched[i]] = name

/home/adrien/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Now we merge the rows of ´qs_unmatched´ and ´the_unmatched´ DataFrames that matched and we append the merged DataFrame to the ´perfect_match_merged´ one.

In [1598]:
new_df = pd.merge(qs_unmatched[qs_columns].iloc[indices_qs_matched].copy(),\
                  the_unmatched[the_columns].iloc[indices_the_matched].copy(), how="outer")
perfect_match_merged = perfect_match_merged.append(new_df)

In [1599]:
the_unmatched = the_unmatched.drop(the_unmatched.index[indices_the_matched])
qs_unmatched = qs_unmatched.drop(qs_unmatched.index[indices_qs_matched])

names_the_unmatched = [names_the_unmatched[x] for x in range(len(names_the_unmatched)) if x not in indices_the_matched]
names_qs_unmatched = [names_qs_unmatched[x] for x in range(len(names_qs_unmatched)) if x not in indices_qs_matched]

Now we update the rows that matched. First we set the THE university names that matched to their counterpart university name in the QS unmatched DataFrame. And then we merge those lines together and we append them to the perfect_match_merged DataFrame.

###### Matches for Levenshtein distance less than 6
Now we are going to do the procedure again but with a cut_off of 6 for the Levenshtein distance.

In [1600]:
changes = []
cut_off = 6
for ind_the, name_the in enumerate(names_the_unmatched):
    min_dist = math.inf
    flag = False
    true_the_name = the_unmatched['Name'].iloc[ind_the]
    for ind_qs, name_qs in enumerate(names_qs_unmatched):
        lev_dist = levenshtein_distance(name_qs, name_the)
        if (lev_dist < min_dist and lev_dist < cut_off):
            flag = True
            best_qs = ind_qs
            true_qs_name = qs_unmatched['Name'].iloc[ind_qs]
            min_dist = lev_dist
            #the_unmatched['Name'].loc[ind_the] = true_qs_name
    if (flag == True):
        matched_qs += [m_qs]
        matched_the += [ind_the]
        print(true_the_name, 'with', true_qs_name)
        changes += [[(ind_the, true_the_name), (best_qs, true_qs_name)]]
changes

University of Tubingen with University of Bergen
Korea Advanced Institute of Science and Technology with KAIST - Korea Advanced Institute of Science & Technology
University of Bonn with University of Bath
Indiana University with Osaka University
University of Sussex with University of Bergen
TU Dresden with TU Berlin
Ulm University with Osaka University
Tufts University with Osaka University
University of Miami with University of Otago
University of Dundee with University of Bergen


[[(9, 'University of Tubingen'), (31, 'University of Bergen')],
 [(10, 'Korea Advanced Institute of Science and Technology'),
  (1, 'KAIST - Korea Advanced Institute of Science & Technology')],
 [(11, 'University of Bonn'), (29, 'University of Bath')],
 [(13, 'Indiana University'), (5, 'Osaka University')],
 [(24, 'University of Sussex'), (31, 'University of Bergen')],
 [(27, 'TU Dresden'), (23, 'TU Berlin')],
 [(28, 'Ulm University'), (5, 'Osaka University')],
 [(37, 'Tufts University'), (5, 'Osaka University')],
 [(43, 'University of Miami'), (25, 'University of Otago')],
 [(44, 'University of Dundee'), (31, 'University of Bergen')]]

In [1601]:
indices_the_matched = [10]
indices_qs_matched = [1]

new_the_names = qs_unmatched['Name'].iloc[indices_qs_matched]
for i, name in enumerate(new_the_names):
    the_unmatched['Name'].iloc[indices_the_matched[i]] = name


/home/adrien/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Now we merge the rows of ´qs_unmatched´ and ´the_unmatched´ DataFrames that matched and we append the merged DataFrame to the ´perfect_match_merged´ one.

In [1602]:
new_df = pd.merge(qs_unmatched[qs_columns].iloc[indices_qs_matched].copy(),\
                  the_unmatched[the_columns].iloc[indices_the_matched].copy(), how="outer")
perfect_match_merged = perfect_match_merged.append(new_df)

In [1603]:
the_unmatched = the_unmatched.drop(the_unmatched.index[indices_the_matched])
qs_unmatched = qs_unmatched.drop(qs_unmatched.index[indices_qs_matched])

names_the_unmatched = [names_the_unmatched[x] for x in range(len(names_the_unmatched)) if x not in indices_the_matched]
names_qs_unmatched = [names_qs_unmatched[x] for x in range(len(names_qs_unmatched)) if x not in indices_qs_matched]

###### Matches for Levenshtein distance less than 7
Now we are going to do the procedure again but with a cut_off of 6 for the Levenshtein distance.

In [1604]:
changes = []
cut_off = 7
for ind_the, name_the in enumerate(names_the_unmatched):
    min_dist = math.inf
    flag = False
    true_the_name = the_unmatched['Name'].iloc[ind_the]
    for ind_qs, name_qs in enumerate(names_qs_unmatched):
        lev_dist = levenshtein_distance(name_qs, name_the)
        if (lev_dist < min_dist and lev_dist < cut_off):
            flag = True
            best_qs = ind_qs
            true_qs_name = qs_unmatched['Name'].iloc[ind_qs]
            min_dist = lev_dist
            #the_unmatched['Name'].loc[ind_the] = true_qs_name
    if (flag == True):
        matched_qs += [m_qs]
        matched_the += [ind_the]
        print(true_the_name, 'with', true_qs_name)
        changes += [[(ind_the, true_the_name), (best_qs, true_qs_name)]]
changes

ETH Zurich - Swiss Federal Institute of Technology Zurich with ETH Zurich - Swiss Federal Institute of Technology
University of Freiburg with University of Reading
University of Tubingen with University of Bergen
University of Bonn with University of Bath
Indiana University with Osaka University
Trinity College Dublin with University College Dublin
University of Mannheim with University of Bath
University of Cologne with University of Otago
University of Sussex with University of Bergen
TU Dresden with TU Berlin
Ulm University with Osaka University
University of Arizona with University of Otago
Tufts University with Osaka University
University of Munster with University of Bath
Paris-Sud University with Kyushu University
University of Miami with University of Otago
University of Dundee with University of Bergen
Tilburg University with Tohoku University


[[(0, 'ETH Zurich - Swiss Federal Institute of Technology Zurich'),
  (0, 'ETH Zurich - Swiss Federal Institute of Technology')],
 [(6, 'University of Freiburg'), (45, 'University of Reading')],
 [(9, 'University of Tubingen'), (30, 'University of Bergen')],
 [(10, 'University of Bonn'), (28, 'University of Bath')],
 [(12, 'Indiana University'), (4, 'Osaka University')],
 [(13, 'Trinity College Dublin'), (32, 'University College Dublin')],
 [(17, 'University of Mannheim'), (28, 'University of Bath')],
 [(21, 'University of Cologne'), (24, 'University of Otago')],
 [(23, 'University of Sussex'), (30, 'University of Bergen')],
 [(26, 'TU Dresden'), (22, 'TU Berlin')],
 [(27, 'Ulm University'), (4, 'Osaka University')],
 [(30, 'University of Arizona'), (24, 'University of Otago')],
 [(36, 'Tufts University'), (4, 'Osaka University')],
 [(38, 'University of Munster'), (28, 'University of Bath')],
 [(41, 'Paris-Sud University'), (18, 'Kyushu University')],
 [(42, 'University of Miami'), (24

In [1605]:
indices_the_matched = [0, 13]
indices_qs_matched = [0, 32]

new_the_names = qs_unmatched['Name'].iloc[indices_qs_matched]
for i, name in enumerate(new_the_names):
    the_unmatched['Name'].iloc[indices_the_matched[i]] = name

/home/adrien/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Now we merge the rows of ´qs_unmatched´ and ´the_unmatched´ DataFrames that matched and we append the merged DataFrame to the ´perfect_match_merged´ one.

In [1606]:
new_df = pd.merge(qs_unmatched[qs_columns].iloc[indices_qs_matched].copy(),\
                  the_unmatched[the_columns].iloc[indices_the_matched].copy(), how="outer")
perfect_match_merged = perfect_match_merged.append(new_df)

In [1607]:
the_unmatched = the_unmatched.drop(the_unmatched.index[indices_the_matched])
qs_unmatched = qs_unmatched.drop(qs_unmatched.index[indices_qs_matched])

names_the_unmatched = [names_the_unmatched[x] for x in range(len(names_the_unmatched)) if x not in indices_the_matched]
names_qs_unmatched = [names_qs_unmatched[x] for x in range(len(names_qs_unmatched)) if x not in indices_qs_matched]

###### Matches for Levenshtein distance less than 8
Now we are going to do the procedure again but with a cut_off of 6 for the Levenshtein distance.

In [1608]:
changes = []
cut_off = 8
for ind_the, name_the in enumerate(names_the_unmatched):
    min_dist = math.inf
    flag = False
    true_the_name = the_unmatched['Name'].iloc[ind_the]
    for ind_qs, name_qs in enumerate(names_qs_unmatched):
        lev_dist = levenshtein_distance(name_qs, name_the)
        if (lev_dist < min_dist and lev_dist < cut_off):
            flag = True
            best_qs = ind_qs
            true_qs_name = qs_unmatched['Name'].iloc[ind_qs]
            min_dist = lev_dist
            #the_unmatched['Name'].loc[ind_the] = true_qs_name
    if (flag == True):
        matched_qs += [m_qs]
        matched_the += [ind_the]
        print(true_the_name, 'with', true_qs_name)
        changes += [[(ind_the, true_the_name), (best_qs, true_qs_name)]]
changes

University of Freiburg with University of Reading
University of Tubingen with University of Bergen
University of Bonn with University of Bath
Vanderbilt University with Yonsei University
Indiana University with Osaka University
Georgetown University with Korea University
University of Mannheim with University of Bath
University of Cologne with University of Otago
University of Sussex with University of Bergen
University of Notre Dame with University of Otago
TU Dresden with TU Berlin
Ulm University with Osaka University
University of Leicester with University of Bergen
University of Arizona with University of Otago
University of Wurzburg with University of Waterloo
Tufts University with Osaka University
University of Munster with University of Bath
Paris-Sud University with Kyushu University
University of Miami with University of Otago
University of Dundee with University of Bergen
University of East Anglia with University of Waterloo
Tilburg University with Tohoku University


[[(5, 'University of Freiburg'), (43, 'University of Reading')],
 [(8, 'University of Tubingen'), (29, 'University of Bergen')],
 [(9, 'University of Bonn'), (27, 'University of Bath')],
 [(10, 'Vanderbilt University'), (10, 'Yonsei University')],
 [(11, 'Indiana University'), (3, 'Osaka University')],
 [(13, 'Georgetown University'), (9, 'Korea University')],
 [(15, 'University of Mannheim'), (27, 'University of Bath')],
 [(19, 'University of Cologne'), (23, 'University of Otago')],
 [(21, 'University of Sussex'), (29, 'University of Bergen')],
 [(22, 'University of Notre Dame'), (23, 'University of Otago')],
 [(24, 'TU Dresden'), (21, 'TU Berlin')],
 [(25, 'Ulm University'), (3, 'Osaka University')],
 [(27, 'University of Leicester'), (29, 'University of Bergen')],
 [(28, 'University of Arizona'), (23, 'University of Otago')],
 [(32, 'University of Wurzburg'), (24, 'University of Waterloo')],
 [(34, 'Tufts University'), (3, 'Osaka University')],
 [(36, 'University of Munster'), (27, 

We don't have any matches any longer so we are just going to assume that all the remaining universities in the two DataFrames of unmatched universities are just universities that are in only one of the two rankings. Therefore all these extra universities will be added to the general DataFrame that we are growing.

In [1609]:
perfect_match_merged.tail(20)

,Location,Name,QS No International Staffs,QS No International Students,QS No Staffs,QS No Students,QS Rank,QS Score,Region,THE International Students Ratio,THE No Students,THE No Students per Staff,THE Rank,THE Score
179,Netherlands,University of Twente,299.0,2235.0,910.0,9332.0,179.0,49.7,Europe,27%,"6,697",12.8,=179,53.8
180,Germany,University of Göttingen,641.0,3736.0,3678.0,30402.0,181.0,49.6,Europe,12%,"28,554",27.9,=113,59.2
183,United States,University of Colorado Boulder,566.0,2679.0,3685.0,30219.0,182.0,49.5,North America,10%,"30,780",16.3,=100,60.5
185,United States,University of Rochester,488.0,2805.0,2569.0,9636.0,186.0,49.3,North America,29%,"9,636",4.3,=153,55.6
190,South Africa,University of Cape Town,379.0,3325.0,1733.0,19593.0,191.0,48.9,Africa,18%,"20,775",11.7,171,54.4
191,Italy,Scuola Normale Superiore di Pisa,1.0,48.0,86.0,532.0,192.0,48.5,Europe,7%,515,5.2,184,53.4
194,Sweden,Stockholm University,646.0,2636.0,2154.0,28281.0,195.0,48.1,Europe,8%,"33,156",24.9,=134,57.5
196,United States,Texas A&M University,206.0,4900.0,3446.0,60294.0,195.0,48.1,North America,9%,"56,245",23.6,=159,55.0
198,Netherlands,Maastricht University,502.0,8234.0,1277.0,16385.0,200.0,47.9,Europe,50%,"16,727",18.0,103,60.4
0,United States,Washington University in St. Louis,784.0,2706.0,2699.0,13474.0,100.0,64.6,North America,17%,"12,600",7.5,=50,70.8


In [1610]:
print(names_the_unmatched)
print()
print(names_qs_unmatched)

['LMUMunich', 'KarolinskaInstitute', 'HeidelbergUniversity', 'WageningenUniversityResearch', 'ParisSciencesLettresPSLResearchUniversityParis', 'UniversityFreiburg', 'UniversityNewSouthWales', 'TechnicalUniversityBerlin', 'UniversityTubingen', 'UniversityBonn', 'VanderbiltUniversity', 'IndianaUniversity', 'RadboudUniversityNijmegen', 'GeorgetownUniversity', 'PierreMarieCurieUniversity', 'UniversityMannheim', 'ArizonaStateUniversity', 'ChariteUniversitaetsmedizinBerlin', 'PompeuFabraUniversity', 'UniversityCologne', 'AutonomousUniversityBarcelona', 'UniversitySussex', 'UniversityNotreDame', 'ScuolaSuperioreSantAnna', 'TUDresden', 'UlmUniversity', 'CaseWesternReserveUniversity', 'UniversityLeicester', 'UniversityArizona', 'UniversityCaliforniaSantaCruz', 'UniversityErlangenNuremberg', 'VrijeUniversiteitAmsterdam', 'UniversityWurzburg', 'UniversityAlabamaBirmingham', 'TuftsUniversity', 'RutgerstheStateUniversityNewJersey', 'UniversityMunster', 'UniversiteLibreBruxelles', 'UniversityLuxembo

###### Appending remaining unmatched universities to ´perfect_match_merged´

In [1611]:
print(perfect_match_merged.shape)
print(the_unmatched.shape)
print(qs_unmatched.shape)

(150, 14)
(50, 14)
(50, 14)


In [1612]:
merged_uni_rank = perfect_match_merged.append(the_unmatched)
merged_uni_rank = merged_uni_rank.append(qs_unmatched)
print(merged_uni_rank.shape)

(250, 14)


In [1613]:
merged_uni_rank.tail(70)

,Location,Name,QS No International Staffs,QS No International Students,QS No Staffs,QS No Students,QS Rank,QS Score,Region,THE International Students Ratio,THE No Students,THE No Students per Staff,THE Rank,THE Score
241,Germany,University of Erlangen-Nuremberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12%,"38,730",58.5,=162,54.8
242,Netherlands,Vrije Universiteit Amsterdam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7%,"24,554",18.1,=165,54.7
243,Germany,University of Wurzburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9%,"22,537",33.9,=165,54.7
244,United States,University of Alabama at Birmingham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6%,"15,703",15.2,168,54.6
245,United States,Tufts University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16%,"10,463",9.3,=169,54.5
246,United States,"Rutgers, the State University of New Jersey",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10%,"43,439",11.1,172,54.3
247,Germany,University of Munster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7%,"42,287",42.6,=173,54.2
248,Belgium,Universite Libre de Bruxelles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35%,"25,343",43.4,=175,54.0
249,Luxembourg,University of Luxembourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57%,"4,969",14.6,=179,53.8
250,France,Paris-Sud University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16%,"27,221",14.5,181,53.7


In [1614]:
merged_uni_rank['Name'].value_counts()

Australian National University                      1
University of St Andrews                            1
Paris-Sud University                                1
London School of Economics and Political Science    1
University of Virginia                              1
University of Notre Dame                            1
Georgia Institute of Technology                     1
University of California, Santa Cruz                1
Vrije Universiteit Amsterdam                        1
Hebrew University of Jerusalem                      1
University of Chicago                               1
Maastricht University                               1
University of Alabama at Birmingham                 1
Ohio State University                               1
University of Bergen                                1
RWTH Aachen University                              1
University of Washington                            1
Aalto University                                    1
Indian Institute of Technolo

We now set the ´Name´ field as an index of our ´merged_uni_rank´ DataFrame.

In [1615]:
merged_uni_rank = merged_uni_rank.set_index('Name')

In [1619]:
merged_uni_rank_cols = merged_uni_rank.columns.tolist()
rearranged_merged_uni_rank_cols = ['Location', 'Region', 'QS Rank', 'QS Score', 'THE Rank', 'THE Score',\
                        'QS No International Staffs', 'QS No International Students', 'QS No Students',\
                        'QS No Staffs', 'THE International Students Ratio', 'THE No Students',\
                        'THE No Students per Staff']

In [1620]:
merged_uni_rank = merged_uni_rank[rearranged_merged_uni_rank_cols]

In [1621]:
merged_uni_rank.tail(70)

,Location,Region,QS Rank,QS Score,THE Rank,THE Score,QS No International Staffs,QS No International Students,QS No Students,QS No Staffs,THE International Students Ratio,THE No Students,THE No Students per Staff
Name,,,,,,,,,,,,,
University of Erlangen-Nuremberg,Germany,NaN,NaN,NaN,=162,54.8,NaN,NaN,NaN,NaN,12%,"38,730",58.5
Vrije Universiteit Amsterdam,Netherlands,NaN,NaN,NaN,=165,54.7,NaN,NaN,NaN,NaN,7%,"24,554",18.1
University of Wurzburg,Germany,NaN,NaN,NaN,=165,54.7,NaN,NaN,NaN,NaN,9%,"22,537",33.9
University of Alabama at Birmingham,United States,NaN,NaN,NaN,168,54.6,NaN,NaN,NaN,NaN,6%,"15,703",15.2
Tufts University,United States,NaN,NaN,NaN,=169,54.5,NaN,NaN,NaN,NaN,16%,"10,463",9.3
"Rutgers, the State University of New Jersey",United States,NaN,NaN,NaN,172,54.3,NaN,NaN,NaN,NaN,10%,"43,439",11.1
University of Munster,Germany,NaN,NaN,NaN,=173,54.2,NaN,NaN,NaN,NaN,7%,"42,287",42.6
Universite Libre de Bruxelles,Belgium,NaN,NaN,NaN,=175,54.0,NaN,NaN,NaN,NaN,35%,"25,343",43.4
University of Luxembourg,Luxembourg,NaN,NaN,NaN,=179,53.8,NaN,NaN,NaN,NaN,57%,"4,969",14.6


In [1622]:
merged_uni_rank.shape

(250, 13)

In [1623]:
merged_uni_rank.to_csv("merged_uni_rank.csv")